# Notes

For how to calculate area under the curve: 
https://stackoverflow.com/questions/13320262/calculating-the-area-under-a-curve-given-a-set-of-coordinates-without-knowing-t


other ideas for characterizing data: 

-Fourier Transform 

-Positive vs Negative area under curve (calculate baseline value, then then spikes beneath it are negative area, e.g. trough area

-Peak heights and distances 

-sincNet: probably not that effective given that its setup for human voice, not ECG 

-wavelet transforms: 
    -likely to be better than fourier transform for non-sinusoidal shape 
    -mostly  used in sincnet? 
    -Droplet Wave transform may be very useful
    
-Spectrum Analysis: Seems to not be that great according to the first paper below, maybe not worth it


Filtering: 
    -would likely help if needed, but I think the dataset has already had soome filtering applied 
    -would need to run some computations for our specific dataset to determine frequencies to include/exclude in a band-pass filter 


TERMA 
    -would need to know the approximate cycle time between each heartbeat as well as a lot of other parameters
    -we may just be able to use the mdpi calculated frequencies for the TERMA approach and assume they're pretty good, given that it's working to solve the exact same problem 
    -setting up TERMA to pick out features shouldn't be too crazy but is definitely a good amount of work on its own 
    -according to study below, is simpler than other wave detectors and gets better results on most things 
   -requires good signal to noise ratio (SNR)
    
    
    
    
OTHER WAVE DETECTORS: a whole bunch are listed for comparison in the testing results section of the mdpi study below



# Studies 

softmax, ReLU(Leaky ReLU), CNN, SincNet, mentions wavelet transforms
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8469424/

goes into the different aspects of the waves themselves more: 
https://www.nature.com/articles/s41598-021-97118-5 

Two event-related moving averages (TERMA - method for detecting R waves and other important ECG characteristics): 
https://www.mdpi.com/2079-6374/6/4/55/htm

A comparison of a whole bunch of different models 
https://arxiv.org/pdf/2004.13701.pdf


    
    


In [7]:
!pip install wfdb


In [25]:
##SAMPLE CODE FROM DATASET INFO PAGE - IMPORTS DATA AND SPLITS INTO TEST/TRAIN SET

import pandas as pd
import numpy as np
import wfdb
import ast

def load_raw_data(df, sampling_rate, path):
    #choose 100 for the 100hz file (500mb) and 500hz (2.7gB)
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = r"C:\Users\scrav\Documents\ECE 493R\FInal Project - EKG\PBT-XL Extracted\\"


# load and convert annotation data
Y = pd.read_csv(r"C:\Users\scrav\Documents\ECE 493R\FInal Project - EKG\PBT-XL Extracted\ptbxl_database.csv", index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

In [38]:
print()

KeyError: 10